In [1]:
import sys
sys.path.append('/home/iceberg/notebooks/PyCon_LT_Workshop')

from helpers.utils import get_spark_session, get_yellow_taxi_data
spark = get_spark_session("getting_started")


24/02/28 18:36:34 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## FILES Metadata

In [37]:
%%sql
SELECT * FROM staging.yellow_taxi.files

content,file_path,file_format,spec_id,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id,readable_metrics
0,s3://warehouse/staging/yellow_taxi/data/00000-1-53970bbb-7371-4dba-882f-ff226d23679a-00001.parquet,PARQUET,0,2463931,38145266,"{1: 317094, 2: 9339593, 3: 9518277, 4: 504049, 5: 3415580, 6: 171197, 7: 98668, 8: 1745483, 9: 2415756, 10: 366600, 11: 2571248, 12: 588823, 13: 75729, 14: 2703679, 15: 266518, 16: 45767, 17: 3501333, 18: 227463, 19: 189633}","{1: 2463931, 2: 2463931, 3: 2463931, 4: 2463931, 5: 2463931, 6: 2463931, 7: 2463931, 8: 2463931, 9: 2463931, 10: 2463931, 11: 2463931, 12: 2463931, 13: 2463931, 14: 2463931, 15: 2463931, 16: 2463931, 17: 2463931, 18: 2463931, 19: 2463931}","{1: 0, 2: 0, 3: 0, 4: 71503, 5: 0, 6: 71503, 7: 71503, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 71503, 19: 71503}","{16: 0, 17: 0, 18: 0, 19: 0, 4: 0, 5: 0, 6: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0}","{1: bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b""@\r\x14\'__\x04\x00""), 3: bytearray(b'\x00\xe4\xa8\xc3__\x04\x00'), 4: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 5: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 6: bytearray(b'\x00\x00\x00\x00\x00\x00\xf0?'), 7: bytearray(b'N'), 8: bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00'), 9: bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00'), 10: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 11: bytearray(b'\x00\x00\x00\x00\x00\x00~\xc0'), 12: bytearray(b'\x00\x00\x00\x00\x00\x00\x12\xc0'), 13: bytearray(b'\x00\x00\x00\x00\x00\x00\xe0\xbf'), 14: bytearray(b'\xaeG\xe1z\x14N_\xc0'), 15: bytearray(b'ffffff?\xc0'), 16: bytearray(b'333333\xd3\xbf'), 17: bytearray(b'\xcd\xcc\xcc\xcc\xcc\x04~\xc0'), 18: bytearray(b'\x00\x00\x00\x00\x00\x00\x04\xc0'), 19: bytearray(b'\x00\x00\x00\x00\x00\x00\xf4\xbf')}","{1: bytearray(b'\x06\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'@\xef\xcaIO\xdf\x05\x00'), 3: bytearray(b'@\xfe\x88^O\xdf\x05\x00'), 4: bytearray(b'\x00\x00\x00\x00\x00\x00""@'), 5: bytearray(b'\xecQ\xb8\x1e\xbd\xaf\x12A'), 6: bytearray(b'\x00\x00\x00\x00\x00\xc0X@'), 7: bytearray(b'Y'), 8: bytearray(b'\t\x01\x00\x00\x00\x00\x00\x00'), 9: bytearray(b'\t\x01\x00\x00\x00\x00\x00\x00'), 10: bytearray(b'\x05\x00\x00\x00\x00\x00\x00\x00'), 11: bytearray(b'{\x14\xaeG\x11{\x18A'), 12: bytearray(b'\x00\x00\x00\x00\x00\xc0@@'), 13: bytearray(b'\xd7\xa3p=\n\x970@'), 14: bytearray(b'\xd7\xa3p=\n\xc7\x8b@'), 15: bytearray(b'\x9a\x99\x99\x99\x99)h@'), 16: bytearray(b'333333\xd3?'), 17: bytearray(b'\xaeG\xe1z\x1e{\x18A'), 18: bytearray(b'\x00\x00\x00\x00\x00\x00\x04@'), 19: bytearray(b'\x00\x00\x00\x00\x00\x00\xf4?')}",None,[4],None,0,"Row(DOLocationID=Row(column_size=2415756, value_count=2463931, null_value_count=0, nan_value_count=None, lower_bound=1, upper_bound=265), PULocationID=Row(column_size=1745483, value_count=2463931, null_value_count=0, nan_value_count=None, lower_bound=1, upper_bound=265), RatecodeID=Row(column_size=171197, value_count=2463931, null_value_count=71503, nan_value_count=0, lower_bound=1.0, upper_bound=99.0), VendorID=Row(column_size=317094, value_count=2463931, null_value_count=0, nan_value_count=None, lower_bound=1, upper_bound=6), airport_fee=Row(column_size=189633, value_count=2463931, null_value_count=71503, nan_value_count=0, lower_bound=-1.25, upper_bound=1.25), congestion_surcharge=Row(column_size=227463, value_count=2463931, null_value_count=71503, nan_value_count=0, lower_bound=-2.5, upper_bound=2.5), extra=Row(column_size=588823, value_count=2463931, null_value_count=0, nan_value_count=0, lower_bound=-4.5, upper_bound=33.5), fare=Row(column_size=2571248, value_count=2463931, null_value_count=0, nan_value_count=0, lower_bound=-480.0, upper_bound=401092.32), improvement_surcharge=Row(column_size=45767, value_count=2463931, null_value_count=0, nan_value_count=0, lower_bound=-0.3, upper_bound=0.3), mta_tax=Row(co

In [43]:
%%sql
select file_path, file_size_in_bytes from staging.yellow_taxi.files

file_path,file_size_in_bytes
s3://warehouse/staging/yellow_taxi/data/00000-1-53970bbb-7371-4dba-882f-ff226d23679a-00001.parquet,38145266
s3://warehouse/staging/yellow_taxi/data/00001-2-53970bbb-7371-4dba-882f-ff226d23679a-00001.parquet,45701454


In [45]:
%%sql
ALTER TABLE staging.yellow_taxi UNSET TBLPROPERTIES ('write.target-file-size-bytes')

++
||
++
++

In [46]:
%%sql
CALL system.rewrite_data_files(table => 'staging.yellow_taxi', options => map('target-file-size-bytes','12800000'))

rewritten_data_files_count,added_data_files_count,rewritten_bytes_count,failed_data_files_count
2,6,83846720,0


In [47]:
%%sql
select file_path, file_size_in_bytes from staging.yellow_taxi.files

file_path,file_size_in_bytes
s3://warehouse/staging/yellow_taxi/data/00000-627-d57d17c1-ca14-4de5-afa4-c63fb78ea005-00002.parquet,17790538
s3://warehouse/staging/yellow_taxi/data/00001-628-d57d17c1-ca14-4de5-afa4-c63fb78ea005-00002.parquet,17679889
s3://warehouse/staging/yellow_taxi/data/00001-628-d57d17c1-ca14-4de5-afa4-c63fb78ea005-00001.parquet,17707227
s3://warehouse/staging/yellow_taxi/data/00001-628-d57d17c1-ca14-4de5-afa4-c63fb78ea005-00003.parquet,10771834
s3://warehouse/staging/yellow_taxi/data/00000-627-d57d17c1-ca14-4de5-afa4-c63fb78ea005-00001.parquet,17710577
s3://warehouse/staging/yellow_taxi/data/00000-627-d57d17c1-ca14-4de5-afa4-c63fb78ea005-00003.parquet,3610122


## Expiring Snapshots

In [6]:
%%sql
select * from staging.yellow_taxi

24/02/28 18:37:05 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.8,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.1,1.0,N,236,42,1,8.0,0.5,0.5,4.0,0.0,0.3,13.3,0.0,0.0
2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.0,0.0,0.3,11.8,2.5,0.0
2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.3,1.0,N,68,163,1,23.5,0.5,0.5,3.0,0.0,0.3,30.3,2.5,0.0
1,2022-01-01 00:40:15,2022-01-01 01:09:48,1.0,10.3,1.0,N,138,161,1,33.0,3.0,0.5,13.0,6.55,0.3,56.35,2.5,0.0
2,2022-01-01 00:20:50,2022-01-01 00:34:58,1.0,5.07,1.0,N,233,87,1,17.0,0.5,0.5,5.2,0.0,0.3,26.0,2.5,0.0
2,2022-01-01 00:13:04,2022-01-01 00:22:45,1.0,2.02,1.0,N,238,152,2,9.0,0.5,0.5,0.0,0.0,0.3,12.8,2.5,0.0
2,2022-01-01 00:30:02,2022-01-01 00:44:49,1.0,2.71,1.0,N,166,236,1,12.0,0.5,0.5,2.25,0.0,0.3,18.05,2.5,0.0
2,2022-01-01 00:48:52,2022-01-01 00:53:28,1.0,0.78,1.0,N,236,141,2,5.0,0.5,0.5,0.0,0.0,0.3,8.8,2.5,0.0


In [7]:
%%sql
select * from staging.yellow_taxi.snapshots

committed_at,snapshot_id,parent_id,operation,manifest_list,summary
2024-02-28 18:11:12.119000,7958142262214527681,None,append,s3://warehouse/staging/yellow_taxi/metadata/snap-7958142262214527681-1-d4980ab5-53cf-4c0c-b013-84835c6e022e.avro,"{'spark.app.id': 'local-1709143859116', 'changed-partition-count': '1', 'added-data-files': '2', 'total-equality-deletes': '0', 'added-records': '5443362', 'total-position-deletes': '0', 'added-files-size': '83846720', 'total-delete-files': '0', 'total-files-size': '83846720', 'total-records': '5443362', 'total-data-files': '2'}"
2024-02-28 18:12:42.384000,2204675250161940923,None,append,s3://warehouse/staging/yellow_taxi/metadata/snap-2204675250161940923-1-ce1d6301-96ad-48d3-84d8-d121eeb001d2.avro,"{'spark.app.id': 'local-1709143924512', 'changed-partition-count': '1', 'added-data-files': '2', 'total-equality-deletes': '0', 'added-records': '5443362', 'total-position-deletes': '0', 'added-files-size': '83846720', 'total-delete-files': '0', 'total-files-size': '83846720', 'total-records': '5443362', 'total-data-files': '2'}"
2024-02-28 18:16:38.868000,8135020296887492846,2204675250161940923,delete,s3://warehouse/staging/yellow_taxi/metadata/snap-8135020296887492846-1-edc0ad69-c32a-42ec-a54b-c682ed375384.avro,"{'spark.app.id': 'local-1709143924512', 'changed-partition-count': '0', 'total-equality-deletes': '0', 'total-position-deletes': '0', 'total-delete-files': '0', 'total-files-size': '83846720', 'total-records': '5443362', 'total-data-files': '2'}"
2024-02-28 18:31:30.922000,3269703635560486185,2204675250161940923,delete,s3://warehouse/staging/yellow_taxi/metadata/snap-3269703635560486185-1-02af94c6-1b60-4102-b63a-1fcb48fab57b.avro,"{'spark.app.id': 'local-1709143924512', 'changed-partition-count': '0', 'total-equality-deletes': '0', 'total-position-deletes': '0', 'total-delete-files': '0', 'total-files-size': '83846720', 'total-records': '5443362', 'total-data-files': '2'}"
2024-02-28 18:32:08.686000,960213915591769294,3269703635560486185,overwrite,s3://warehouse/staging/yellow_taxi/metadata/snap-960213915591769294-1-0a18135e-eda0-4c48-bc98-7db617c80048.avro,"{'added-data-files': '2', 'total-equality-deletes': '0', 'added-records': '5412306', 'deleted-data-files': '2', 'deleted-records': '5443362', 'total-records': '5412306', 'spark.app.id': 'local-1709143924512', 'removed-files-size': '83846720', 'changed-partition-count': '1', 'total-position-deletes': '0', 'added-files-size': '82967139', 'total-delete-files': '0', 'total-files-size': '82967139', 'total-data-files': '2'}"


In [8]:
from minio import Minio
client = Minio(
        "minio:9000",
        access_key="admin",
        secret_key="password",
        secure=False  # Set to True if your Minio server uses TLS/SSL
    )

In [9]:
def print_objects_in_warehouse(bucket_name):
    objects = client.list_objects(bucket_name, recursive=True)
    for obj in objects:
        print(obj.object_name, str(obj._size//1024) + "KB")
print_objects_in_warehouse("warehouse")

staging/yellow_taxi/data/00000-1-53970bbb-7371-4dba-882f-ff226d23679a-00001.parquet 37251KB
staging/yellow_taxi/data/00000-1-f2196937-4eab-440c-b1b3-6d36117bf0b3-00001.parquet 37251KB
staging/yellow_taxi/data/00000-28-f435be26-8f70-45f9-8bd0-fc38a9a27d8d-00001.parquet 36874KB
staging/yellow_taxi/data/00001-2-53970bbb-7371-4dba-882f-ff226d23679a-00001.parquet 44630KB
staging/yellow_taxi/data/00001-2-f2196937-4eab-440c-b1b3-6d36117bf0b3-00001.parquet 44630KB
staging/yellow_taxi/data/00001-29-f435be26-8f70-45f9-8bd0-fc38a9a27d8d-00001.parquet 44147KB
staging/yellow_taxi/metadata/00000-26f32088-1266-46a5-86b4-b3366abaea5b.metadata.json 3KB
staging/yellow_taxi/metadata/00001-ad69dd8a-9afc-48f6-a01b-ee6617395386.metadata.json 4KB
staging/yellow_taxi/metadata/00002-d89a34de-1e11-47bd-99df-e518de5d88db.metadata.json 6KB
staging/yellow_taxi/metadata/00003-bfe4b03b-d2b1-4711-9c1b-1cde75830bcf.metadata.json 7KB
staging/yellow_taxi/metadata/00004-a1d03948-0da5-45ee-b3b2-8a1e6658d569.metadata.json 

In [29]:
from datetime import datetime, timedelta

now_time = datetime.now()
ten_mins_ago = now_time - timedelta(minutes=10)
formatted_ten_mins_ago = ten_mins_ago.strftime("%Y-%m-%d %H:%M:%S")

In [31]:
spark.sql(f"CALL system.expire_snapshots(table => 'staging.yellow_taxi', older_than => TIMESTAMP '{formatted_ten_mins_ago}', retain_last => 1)").show()

+------------------------+-----------------------------------+-----------------------------------+----------------------------+----------------------------+------------------------------+
|deleted_data_files_count|deleted_position_delete_files_count|deleted_equality_delete_files_count|deleted_manifest_files_count|deleted_manifest_lists_count|deleted_statistics_files_count|
+------------------------+-----------------------------------+-----------------------------------+----------------------------+----------------------------+------------------------------+
|                       4|                                  0|                                  0|                           3|                           4|                             0|
+------------------------+-----------------------------------+-----------------------------------+----------------------------+----------------------------+------------------------------+



In [32]:
print_objects_in_warehouse("warehouse")

staging/yellow_taxi/data/00000-1-53970bbb-7371-4dba-882f-ff226d23679a-00001.parquet 37251KB
staging/yellow_taxi/data/00001-2-53970bbb-7371-4dba-882f-ff226d23679a-00001.parquet 44630KB
staging/yellow_taxi/metadata/00000-26f32088-1266-46a5-86b4-b3366abaea5b.metadata.json 3KB
staging/yellow_taxi/metadata/00001-ad69dd8a-9afc-48f6-a01b-ee6617395386.metadata.json 4KB
staging/yellow_taxi/metadata/00002-d89a34de-1e11-47bd-99df-e518de5d88db.metadata.json 6KB
staging/yellow_taxi/metadata/00003-bfe4b03b-d2b1-4711-9c1b-1cde75830bcf.metadata.json 7KB
staging/yellow_taxi/metadata/00004-a1d03948-0da5-45ee-b3b2-8a1e6658d569.metadata.json 8KB
staging/yellow_taxi/metadata/00005-6bfa6dd0-83a7-4e7e-8e76-ecc11a296bf0.metadata.json 8KB
staging/yellow_taxi/metadata/00006-5a7ae5b2-e409-4d8a-b417-756e585e734b.metadata.json 10KB
staging/yellow_taxi/metadata/00007-91e8d0ba-010f-4bae-955d-f2900f176f02.metadata.json 10KB
staging/yellow_taxi/metadata/00008-f43af439-dc27-46e9-8e34-4d84194b2fe5.metadata.json 7KB
stag

In [33]:
%%sql
select * from staging.yellow_taxi.snapshots

committed_at,snapshot_id,parent_id,operation,manifest_list,summary
2024-02-28 18:12:42.384000,2204675250161940923,None,append,s3://warehouse/staging/yellow_taxi/metadata/snap-2204675250161940923-1-ce1d6301-96ad-48d3-84d8-d121eeb001d2.avro,"{'spark.app.id': 'local-1709143924512', 'changed-partition-count': '1', 'added-data-files': '2', 'total-equality-deletes': '0', 'added-records': '5443362', 'total-position-deletes': '0', 'added-files-size': '83846720', 'total-delete-files': '0', 'total-files-size': '83846720', 'total-records': '5443362', 'total-data-files': '2'}"


## What Does rewrite_manifests Do?
The rewrite_manifests operation in Iceberg is designed to optimize the table by rewriting its manifest files. Manifest files in Iceberg track metadata about data files (like Parquet files) in a table. Over time, as a table is updated, deleted, or appended to, the number of manifest files can grow, and reading these manifests can become inefficient.

The rewrite_manifests operation combats this by consolidating smaller manifest files into larger ones, which can improve query performance by reducing the overhead of reading many small files. This operation can be particularly beneficial after a large number of small updates or deletions.

In [48]:
spark.sql("CALL system.rewrite_manifests('staging.yellow_taxi')").show()

24/02/28 18:50:55 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------------------+---------------------+
|rewritten_manifests_count|added_manifests_count|
+-------------------------+---------------------+
|                        2|                    1|
+-------------------------+---------------------+

